<img src='images/gdd-logo.png' width='300px' align='right' style="padding: 15px">


# Scikit Learn Refresh

This notebook is a refresher on using Scikit-Learn to build a machine learning model.

You will start by building a baseline model and then iterate on it in later notebooks to improve its performance.

- [About the data](#about)
- [<mark>Exercise: Data Exploration</mark>](#explore)
- [Establishing a baseline](#baseline)
- [Generalisation (splitting the data)](#splitting)
- [Model creation and evaluation](#build)
- [Conclusion and next steps](#conc)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

<a id=about></a>
## About the data

<img src='images/who.png' width='500px' align='right' style="padding: 15px">

According to the World Health Organization (WHO), strokes are the 2nd leading cause of death globally, responsible for approximately 11% of total deaths.

You will use this dataset to build a model that can **predict whether a patient is likely to have a `stroke`** (based on input parameters like gender, age and whether or not they smoke). 

Each row in the data provides relavant information about the patient.
### Features

1. `id`: unique identifier
1. `address`: a general address (city/county, state and postal code)
1. `gender`: "Male", "Female" or "Other"
1. `age`: age of the patient
1. `hypertension`: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension
1. `heart_disease`: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease
1. `ever_married`: "No" or "Yes"
1. `work_type`: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"
1. `residence_type`: "Rural" or "Urban"
1. `avg_glucose_level`: average glucose level in blood
1. `bmi`: body mass index
1. `smoking_status`: "formerly smoked", "never smoked", "smokes" or "Unknown"
1. `stroke`: 1 if the patient had a stroke or 0 if not

*Note: "Unknown" in smoking_status means that the information is unavailable for this patient.*

<a id=explore></a>

## Data Exploration

Let's start by importing the data and investigating its features.

In [ ]:
stroke = pd.read_csv('data/stroke.csv').rename(columns=str.lower)
stroke.head()

### <mark>Exercise</mark>

Conduct a preliminary analysis of the data by answering the questions below.

1. Take a look at missing values:
    - a) Are there any missing values in the data?
    - b) Use seaborn's `sns.heatmap` (on `df.isnull()`) to determine if there is a pattern to the missing values.
    - c) How will missing values impact training a machine learning model?

<details>
<summary><span style="color:blue">Show text answers</span></summary>

If a sample contains missing values for one or more features (or the outcome), this observation cannot be used to train the ML model. 

Generally there are two options: 
1) Either you drop the sample entirely, or 
2) you try to fill/interpolate the missing value by using a constant, the median/mean feature value, or prediction using another (ML) model. We will discuss this in detail later.  
  

</details>

*Uncomment cell below to see the code answer:*

In [ ]:
# %load answers/01-data-exploration-1.py

2. Investigate the feature columns:
    - a) What are the datatypes of each feature? 
    - b) Do they make sense?
    - c) How will different types of features impact training a machine learning model?
    - d) Which features do you think will be informative for predicting stroke?

<details>
<summary><span style="color:blue">Show text answers</span></summary>

b) They mostly make sense. In Pandas, the `object` Dtype usually refers to strings. `age` being float64 is odd, since we usually measure age in full years. It would be good to investigate, if age is indeed only in full years or in fractions. Try: `stroke['age'].unique()`<br>
c) Computers cannot interpret strings. In order to use string/text features, you will have to find a way to encode strings into numbers. More about this later.<br>
d) Except the `id` column, all of the features could be potentially informative. However, `gender` and `who` could potentially encode the same information. Try: `stroke['who'].unique()`
</details>

*Uncomment cell below to see the code answer:*

In [ ]:
# %load answers/01-data-exploration-2.py

**Bonus:** Investigate if there are any large outliers in the data. What effect would large outliers have on training a machine learning model? How would you treat them?

<a id=baseline></a>

## Establishing a baseline

In a first step, you will develop a simple baseline model.


<mark>**Question**:</mark> Why is a baseline model useful?

<details>
<summary><span style="color:blue">Show answer</span></summary>
A baseline provides a point of reference from which to compare each model iteration as we build up our model and features. 

Without a baseline, you have **no point of reference** to consider whether you are continuing to add value with more complex features/ models. It defines the **hurdle** that all other machine learning algorithms must cross to demonstrate “skill” on the problem.
</details>


The simplest model would be to always predict the most likely outcome:

In [ ]:
stroke['stroke'].value_counts(
    # normalize = True
    )

In this case, you would always predict no stroke, which would yield a model accuracy of ~95%! This is good, right?

<mark>**Question:**</mark> Why is accuracy a flawed metric in this case?

<details>
<summary><span style="color:blue">Show solution</span></summary>
  
Especially with unbalanced outcomes such as here, high model accuracy does not mean you have a good model, because only ever predicting the majority outcome would already lead to a high accuracy. What you would like the model to do is to make informed predictions. To capture model performance, you will need more refined metrics for this (more about this later).

</details>

Alternatively, you could train a simple machine learning model with minimal preprocessing of your data.

Let's see what the low hanging fruit is by:
- Dropping any categorical (object) columns 
- Dropping columns with missing values
- Dropping the `id` column

<!-- - Building a Decision Tree with a max depth of 2
- Using `class_weight='balanced'` to address the imbalance of data -->

In [ ]:
# dropping columns 

categorical_cols = ['work_type', 'smoking_status', 'who', 'gender', 'residence_type']
missing_cols = ['bmi', 'age']
drop_cols = ['id','address']

target = 'stroke'

X = stroke.drop(columns = drop_cols 
                        + categorical_cols
                        + missing_cols
                        + [target])
y = stroke[target]

In [ ]:
X.head()

In [ ]:
y.head()

<a id=splitting></a>

## Splitting the dataset

In the next step, you will split the dataset into a training set to train the model and a test set assess model performance.

<mark>**Question**:</mark> Why is train-test splitting a good idea?

<details>
<summary><span style="color:blue">Show solution</span></summary>
  
An important goal of machine learning is to create a model that does not only do well on the data that it has already seen, but will also perform well under new circumstances on data that is **has not seen before**. We call this ***generalisation***. 

This why it's import to separate our dataset into two parts:
* The _training_ set: This is the data (features and targets) that will **guide the learning process**. 
* The _test_ set: This is the data (features and targets) that will be used to **evaluate** how well our model has learned on *unseen, new* data. 

</details>

Scikit-learn's `train_test_split()` function allows you to split the data in a train- and test set. By default, the test set size is set to 25% and the data is shuffled. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42, 
                                                    stratify=y)

In [ ]:
X_train.shape, X_test.shape

<mark> **Question:**</mark> What does the parameter `stratify` do and why would you use it?

<details>
<summary><span style="color:blue">Show solution</span></summary>
  
The stratify argument makes sure that the same proportion of stroke to no stroke is present in the training and test set. This is especially important with highly imbalanced data (as the likelihood of an unproportional split is increased). 

</details>

<a id=build></a>

## Model creation and evaluation

You are now ready to build a machine learning model! 

Scikit-learn has a rich [collection of algorithms](https://scikit-learn.org/stable/supervised_learning.html) readily available.

Normally, one model will be sufficient to establish a baseline. However, for educational purposes, we shall explore a few more simple models. 

For each model, the process will be as follows:

#### Scikit-Learn API usage

When training a model, the steps you will need to take in sklearn are always the same:
1. First, choose a model class and import that model 
2. Choose the model hyperparameters by instantiating this class with desired values.
3. Train the model to the preprocessed train data by calling the `fit()` method of the model instance.
4. Evaluate the model's performance using suitable metrics

### Decision Tree Classifier

Let's investigate the performance of a simple model such as the Decision Tree classifier.

Since the data is very imbalanced, you can add `class_weight='balanced'` as an argument.

In [ ]:
# Step 1: Import model
from sklearn.tree import DecisionTreeClassifier

# Step 2: Instantiate model and set parameters
tree_model = DecisionTreeClassifier(max_depth=3, 
                                    class_weight='balanced',
                                    random_state=42)

# Step 3: Train model
tree_model.fit(X_train, y_train)

You already established that accuracy would be a bad metric to use for this particular dataset.

A better metric is the ***Receiver Operating Characteristic - Area Under the Curve*** (ROC-AUC). We will discuss the ROC-AUC more later, but for now: **The higher the ROC-AUC, the better the model** is at predicting 0 classes as 0 and 1 classes as 1. 

Let's see what score you get when using this metric.

In [ ]:
# Step 4: Evaluate model
from sklearn.metrics import roc_auc_score

y_pred_test = tree_model.predict_proba(X_test)[:,1]
y_pred_train = tree_model.predict_proba(X_train)[:,1]

print(f'AUC score: {roc_auc_score(y_test, y_pred_test), roc_auc_score(y_train, y_pred_train)}')

Although the Decision Tree is not necessarily the most high performant model, it is very interpretable. This can be helpful if you need to explain your model to non-technical stakeholders.

In [ ]:
from sklearn.tree import plot_tree

fig,ax = plt.subplots(figsize=(16,4))

plot_tree(tree_model, 
          ax=ax, 
          feature_names=list(X_train.columns), 
          class_names=['no stroke', 'stroke'],
          );

### <mark>Exercise: Logistic regression</mark>

Implement a Logistic Regression model to see how it performs. Make sure to add the `class_weight` parameter again.

In [ ]:
# %load answers/01-logistic_regression.py

## Diving into ROC-AUC

You've used ROC-AUC now as a metric, but how exactly is it calculated?

Let's unpack these questions by starting to look at a confusion matrix.

### Confusion Matrix

A confusion matrix summarises the predictions of a classifier.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(
         tree_model, 
         X_test, y_test,);

<mark>**Question:** For this problem, which numbers would you be most interested in increasing/decreasing? 

<details>
<summary><span style="color:blue">Show solution</span></summary>
  
We would like to see true 0's predicted as 0, and true 1's predicted as 1. Thus, ideally, all predictions should be on the main diagonal of the confusion matrix.

</details>

### ROC and AUC

Typically a binary classifier will use a probability of 0.5 as its threshold for classification.

<img src='images/threshold.png' width='400px' style="padding: 15px">

However, adjusting this threshold may be beneficial for the purpose of your model.

Let's investigate how adjusting the classification threshold affects one of the models you learned previously.

In [ ]:
from custom_threshold import CustomThreshold

thresholds = [0, 0.25, 0.5]

fig, ax = plt.subplots(1,len(thresholds), figsize=(16,8))

for idx, threshold in enumerate(thresholds):

    tree = CustomThreshold(tree_model, threshold)
    
    ConfusionMatrixDisplay.from_estimator(
         tree, 
         X_test, y_test,
         ax=ax[idx], colorbar=False)

The **Receiver Operating Characteristics (ROC) curve**, is a graphical plot which illustrates the performance of a classifier at various threshold settings.

It is created by plotting the **True Positive Rate (TPR)** (the fraction of true positives out of the positives) against the **False Positive Rate (FPR)** (the fraction of false positives out of the negatives), at various threshold settings.

$$TPR = \frac{TP}{TP + FN} \ \ \  \text{vs.} \ \ \ FPR = \frac{FP}{TN + FP}$$

In [ ]:
from sklearn.metrics import RocCurveDisplay

fig, ax = plt.subplots(1, 2, figsize=(16,6))

RocCurveDisplay.from_estimator(tree_model, X_train, y_train, ax=ax[0], name='Train')
RocCurveDisplay.from_estimator(tree_model, X_test, y_test, ax=ax[0], name='Test')
ax[0].set(title='Decision Tree model');

# RocCurveDisplay.from_estimator(logistic_model, X_train, y_train, ax=ax[1], name='Train')
# RocCurveDisplay.from_estimator(logistic_model, X_test, y_test, ax=ax[1], name='Test')
# ax[1].set(title='Logistic model')

Typically, the ideal situation is the top left corner of the plot: a FPR of zero and a TPR of one. 

Whilst this may not be realistic, it does mean that a larger **Area Under the Curve (AUC)** is usually better. The “steepness” of ROC curves is also important, since it is ideal to maximize the TPR while minimizing the FPR.

<mark>**Question:** Why is the AUC score a good metric to use in this use case?</mark>

<details>
  <summary><span style="color:blue">Show solution</span></summary>
    
   AUC is a better metric here due to the imbalanced data issue: ROC analysis does not have any bias toward models that perform well on the minority class at the expense of the majority class - a property that is quite attractive when dealing with imbalanced data.
    
[Page 27, Imbalanced Learning: Foundations, Algorithms, and Applications, 2013](https://www.amazon.com/dp/1118074629/ref=as_li_ss_tl?&linkCode=sl1&tag=inspiredalgor-20&linkId=615e87a9105582e292ad2b7e2c7ea339&language=en_US)

</details>

<mark>**Question:** Why is it important to calculate and visualise the metrics for train and test?</mark>

<details>
  <summary><span style="color:blue">Show solution</span></summary>
    
   It allows you to check for overfitting and whether your model is able to generalise to new data.

</details>

---
<img src='images/gdd-logo.png' width='300px' align='right' style="padding: 15px">

<a id=conc></a>

# Conclusion and next steps

This notebook has covered building a baseline classification problem using the sklearn estimator DecisionTreeClassifier.

Next steps to improve the model:
- Build an ensemble algorithm
- Conduct feature selection
- Generate useful features

Next steps to align with best practice:
- Hypertune model parameters
- Build custom transformers